# **\[FRAUD\]** 그래프자료로 데이터정리

김보람  
2023-08-10

# imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import torch

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics 

# embedding 
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder

In [3]:
def build_graph_bipartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df["cc_num"].values.tolist()+\
                                                      df["merchant"].values.tolist()))}
    
    df["from"]=df["cc_num"].apply(lambda x:mapping[x])  #엣지의 출발점
    df["to"]=df["merchant"].apply(lambda x:mapping[x])  #엣지의 도착점
    
    df = df[['from', 'to', "amt", "is_fraud"]].groupby(['from','to']).agg({"is_fraud":"sum","amt":"sum"}).reset_index()
    df["is_fraud"]=df["is_fraud"].apply(lambda x:1 if x>0 else 0)
    
    G=nx.from_edgelist(df[["from","to"]].values, create_using=graph_type)
    
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["is_fraud"] for idx, x in df[["from","to","is_fraud"]].iterrows()}, "label")  #엣지 속성 설정,각 속성의 사기 여부부     
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["amt"] for idx,x in df[["from","to","amt"]].iterrows()}, "weight") # 엣지 속성 설정, 각 엣지의 거래 금액

    return G


def build_graph_tripartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df.index.values.tolist() + 
                                                       df["cc_num"].values.tolist() +
                                                       df["merchant"].values.tolist()))}
    df["in_node"]= df["cc_num"].apply(lambda x: mapping[x])
    df["out_node"]=df["merchant"].apply(lambda x:mapping[x])
    
        
    G=nx.from_edgelist([(x["in_node"], mapping[idx]) for idx, x in df.iterrows()] +\
                        [(x["out_node"], mapping[idx]) for idx, x in df.iterrows()], create_using=graph_type)
    
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")     
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")   
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")  
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")

    return G
    
    
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def embedding(Graph):
    # Graph -> X (feature)
    _edgs = list(Graph.edges)
    subGraph = Graph.edge_subgraph([_edgs[x] for x in range(len(Graph.edges))]).copy()
    subGraph.add_nodes_from(list(set(Graph.nodes) - set(subGraph.nodes)))    
    embedded = AverageEmbedder(Node2Vec(subGraph, weight_key='weight').fit(window=10).wv)
    X = [embedded[str(_edgs[x][0]), str(_edgs[x][1])] for x in range(len(Graph.edges))]
    # Graph -> y (label)
    y = np.array(list(nx.get_edge_attributes(Graph, "label").values()))
    return X,y 

def anal(df):
    Graph = build_graph_bipartite(df)
    X,XX,y,yy = embedding(Graph)
    lrnr = RandomForestClassifier(n_estimators=100, random_state=42) 
    lrnr.fit(X,y)
    yyhat = lrnr.predict(XX)
    df = pd.DataFrame({
        'acc':[sklearn.metrics.accuracy_score(yy,yyhat)], 
        'pre':[sklearn.metrics.precision_score(yy,yyhat)], 
        'rec':[sklearn.metrics.recall_score(yy,yyhat)],
        'f1':[sklearn.metrics.f1_score(yy,yyhat)]}
    )    
    return df

def our_sampling1(df):
    cus_list = set(df.query('is_fraud==1').cc_num.tolist())
    return df.query("cc_num in @ cus_list")

`-` 모든엣지를 고려

In [19]:
# N = 10 
# edge_index = torch.tensor([[i,j] for i in range(N) for j in range(N)]).T
# # edge_attr = 그래프의 웨이트 

In [20]:
# edge_index

In [4]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [5]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

In [5]:
# diff = fraudTrain.trans_date_trans_time[101]-fraudTrain.trans_date_trans_time[0]

In [6]:
# diff

In [93]:
# diff.total_seconds()

In [55]:
# theta = 86400*1.2
# theta

In [8]:
# theta = 86400*1.2
# np.exp(-diff.total_seconds()/theta)

In [13]:
# !git add .
# !git commit -m. 
# !git push 
# !quarto publish --no-browser --no-prompt

## 해보자

In [20]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

In [25]:
N = len(fraudTrain)
N

`-` 시도1

In [ ]:
edge_index_list = []
for i in range(N):
    for j in range(N):
        time_difference = (fraudTrain['trans_date_trans_time'][i] - fraudTrain['trans_date_trans_time'][j]).total_seconds()
        edge_index_list.append([i, j, time_difference])

In [ ]:
edge_index = torch.tensor(edge_index_list).T

`-` 시도2

In [ ]:
N = len(fraudTrain)
edge_index = torch.tensor([[i,j] for i in range(N) for j in range(N)]).T
# edge_attr = 그래프의 웨이트 

-   너~무 오래걸린다.

`-` 시도3

-   df02을 이용해서 해보자.

In [24]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [12]:
214520*214520

In [16]:
# N = len(df02)
# edge_index = torch.tensor([[i,j] for i in range(N) for j in range(N)]).T

`-` 시도4: df50

In [25]:
df50 = down_sample_textbook(df02)
df50.shape

In [26]:
12012*12012

In [27]:
df50 = df50.reset_index()

In [28]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [74]:
df50_tr.is_fraud.mean().round(5), df50_test.is_fraud.mean().round(5)

In [77]:
df50_tr.shape, df50_test.shape

In [29]:
N = len(df50_tr)
edge_index = torch.tensor([[i,j] for i in range(N) for j in range(N)]).T
edge_index

In [30]:
df50_tr = df50_tr.reset_index()

In [31]:
edge_index_list = []
for i in range(N):
    for j in range(N):
        time_difference = (df50_tr['trans_date_trans_time'][i] - df50_tr['trans_date_trans_time'][j]).total_seconds()
        edge_index_list.append([i, j, time_difference])

In [32]:
edge_index = np.array(edge_index_list)

In [33]:
edge_index.shape

In [34]:
edge_index[:,2] = np.abs(edge_index[:,2])

In [35]:
theta = edge_index[:,2].mean()
theta

In [36]:
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta)!=1) * np.exp(-edge_index[:,2]/theta)

In [37]:
edge_index

In [159]:
eee = edge_index[:,:]

In [161]:
eee[:,1]

In [38]:
edge_index_list_updated = edge_index.tolist()

In [39]:
edge_index_list_updated[:5]

In [53]:
df50_tr

-   cc_num로 그룹별로 묶자.

In [88]:
df50_tr[df50_tr['cc_num']==3.543590e+15]

In [43]:
df50_grouped=df50_tr.groupby(by='cc_num')

In [89]:
edge_index_list = []
for i in range(N):
    for j in range(N):
        if df50_tr['cc_num'][i] != df50_tr['cc_num'][j]:  # cc_num 값이 같다면
            time_difference = 0
        else:
            time_difference = (df50_tr['trans_date_trans_time'][i] - df50_tr['trans_date_trans_time'][j]).total_seconds()
        edge_index_list.append([i, j, time_difference])


In [105]:
edge_index = np.array(edge_index_list)

In [106]:
edge_index.shape

In [107]:
edge_index

In [108]:
edge_index[:,2] = np.abs(edge_index[:,2])

In [109]:
theta = edge_index[:,2].mean()
theta

In [110]:
edge_index

In [111]:
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta)!=1) * np.exp(-edge_index[:,2]/theta)

In [112]:
edge_index

In [113]:
edge_index_list_updated = edge_index.tolist()

In [190]:
np.array(edge_index_list_updated)[:,2].mean()

In [192]:
mm = np.array(edge_index_list_updated)[:,2].mean()

edge_index_list_updated가 w

In [193]:
selected_edges = [(int(row[0]), int(row[1])) for row in edge_index_list_updated if row[2] > mm]

In [194]:
edge_index_selected = torch.tensor(selected_edges, dtype=torch.long).t()

In [197]:
edge_index_selected.shape

In [119]:
x = df50_tr['amt']

In [129]:
x

In [130]:
a = torch.tensor(x, dtype=torch.float)

In [137]:
a = a.reshape(-1,1)
a

In [162]:
y = df50_tr['is_fraud']

In [163]:
b = torch.tensor(y,dtype=torch.int64)

In [164]:
b

In [142]:
import torch_geometric

In [198]:
data = torch_geometric.data.Data(x=a, edge_index = edge_index_selected, y=b)

In [199]:
data

-   예시처럼 edge_index 하려햇ㄴ는뎅.. 흠..

`-` pyg lesson6

In [200]:
gconv = torch_geometric.nn.GCNConv(1,4)
gconv

In [201]:
gconv(data.x, data.edge_index)

In [202]:
list(gconv.parameters())

In [203]:
_,W = list(gconv.parameters())
W

`-` pyg lesson5

In [204]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(9009, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [205]:
model = GCN()

In [206]:
model

In [210]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()

In [209]:
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(data.x, data.y)
    loss.backward()
    optimizer.step()

-   df50_test

In [1]:
df50_test 

In [216]:
data

`x=[9009, 1]` : 9009개의 거래량이 있다. 특징은 걍.. 거래량 하나? \<-
거래자체에 대한 특징을 더 추가해도 좋을듯.. 흠.

`edge_index=[2, 28472]`: 거래끼리의 edge값은 28472.. 일단 고객id가 같지
않으면 엣지값0으로 하고..

In [220]:
data.train_mask